In [1]:
ls -ltrh ./Data/ice

total 7,8M
-rw-rw-r-- 1 test test 2,3M oct 24 13:58 N_seaice_extent_daily_v3.0.csv
-rw-rw-r-- 1 test test 2,3M oct 24 14:03 S_seaice_extent_daily_v3.0.csv
-rw-rw-r-- 1 test test 3,3M oct 25 12:14 Sea_Ice_Index_Regional_Daily_Data_G02135_v3.0.xlsx


In [2]:
ls -ltrh ./Data/co2

total 152
-rwxrwxrwx  1 bogdan  staff    73K Oct 24 12:16 monthly_in_situ_co2_mlo.csv*


In [4]:
ls -ltrh ./Data/fire

total 167374520
-rw-r--r--   1 bogdan  staff   1.4G Oct 24 14:56 fire_archive_M6_82356.json
-rw-r--r--   1 bogdan  staff    71M Oct 24 14:56 fire_nrt_M6_82356.json
-rw-r--r--   1 bogdan  staff   3.0G Oct 24 14:58 fire_archive_V1_82357.json
-rw-r--r--   1 bogdan  staff   1.6G Oct 24 15:01 fire_archive_M6_82363.json
-rw-r--r--   1 bogdan  staff   3.1G Oct 24 15:02 fire_nrt_V1_82357.json
-rw-r--r--   1 bogdan  staff   6.6G Oct 24 15:10 fire_archive_V1_82364.json
-rw-r--r--   1 bogdan  staff   1.7G Oct 25 10:27 fire_archive_M6_82479.json
-rw-r--r--   1 bogdan  staff   6.9G Oct 25 10:36 fire_archive_V1_82480.json
-rw-r--r--   1 bogdan  staff   1.6G Oct 25 10:37 fire_archive_M6_82481.json
-rw-r--r--   1 bogdan  staff   1.8G Oct 25 10:40 fire_archive_M6_82483.json
-rw-r--r--   1 bogdan  staff   7.4G Oct 25 10:49 fire_archive_V1_82484.json
-rw-r--r--   1 bogdan  staff   7.0G Oct 25 10:53 fire_archive_V1_82482.json
-rw-r--r--   1 bogdan  staff   1.6G Oct 25 10:54 fire_archive_M6_82487.json
-rw-

In [1]:
import pandas as pd
import random
import os

The following code was run for all the files, but I am showing how it runs for only one file

In [48]:
fire_files = ['fire_nrt_V1_82357.json']
#fire_files = ['fire_archive_M6_82356.json', 'fire_archive_M6_82363.json', 'fire_archive_M6_82479.json',
#'fire_archive_M6_82481.json', 'fire_archive_M6_82483.json', 'fire_archive_M6_82487.json',
#'fire_archive_M6_82489.json', 'fire_archive_M6_82491.json', 'fire_archive_M6_82493.json',
#'fire_archive_M6_82495.json', 'fire_archive_M6_82497.json', 'fire_archive_M6_82499.json',
#'fire_archive_M6_82501.json', 'fire_archive_M6_82503.json', 'fire_archive_M6_82505.json',
#'fire_archive_M6_82508.json', 'fire_archive_M6_82511.json', 'fire_archive_M6_82513.json',
#'fire_archive_M6_82515.json', 'fire_archive_M6_82517.json', 'fire_archive_V1_82357.json',
#'fire_archive_V1_82364.json', 'fire_archive_V1_82480.json', 'fire_archive_V1_82482.json',
#'fire_archive_V1_82484.json', 'fire_archive_V1_82488.json', 'fire_archive_V1_82490.json',
#'fire_nrt_M6_82356.json', 'fire_nrt_V1_82357.json']
strings = ['"acq_date": ','"acq_time": ','"acq_time": ','"instrument": ',
           '"satellite": ','"version": ','"daynight": ', '"confidence": ']
drops=[]
sample_percentage=0.01
print_progress=100000 # Print '*' after 100000 read observations (about 1000 written observations)

In [49]:
def add_quotation_mark(line, strings, drops):

    for string in drops:
        pos=line.find(string)
        if pos != -1:
            return None

    for string in strings:
        pos=line.find(string)
        if pos != -1:
            index = pos+len(string)
            if line[-2]==',':
                return line[:index]+'"'+line[index:-2]+'"'+',\n'
            else:
                return line[:index]+'"'+line[index:-1]+'"'+'\n' # no comma if it's the last line

    return line

In [50]:
def decision(probability):
    return random.random() < probability # return True/False depending on the the probability

In [51]:
def read_skip_record(input_file, output_file):
    global comma
    observation = ''
    while True:
        json_line = input_file.readline().strip() + '\n'
        processed_line = add_quotation_mark(json_line, strings, drops)
        observation += processed_line
        if processed_line[0] == '}':
            if(decision(sample_percentage)): # write a random "sample_percentage" of the records
                if processed_line[-2] == ',': # last char is '\n'
                    comma = True
                else:
                    comma = False
                output_file.write(observation)
            if processed_line[-2] == ',': # check for comma, last char in line is '\n'
                return True
            else:
                return False

In [52]:
for fire_file in fire_files:
    i = 0
    comma = False
    fire_file = 'Data/fire/' + fire_file
    pos=fire_file.find('.')
    fire_file_out = fire_file[:pos] + '_out' + fire_file[pos:]
    print('\n', fire_file, ' -> ', fire_file_out)

    with open(fire_file, 'r') as file:
        while True:
            with open(fire_file_out, 'w') as file1:
                line = file.readline().strip()
                # '[' on the first line
                if line != '[':
                    print('Wrong json structure!')
                    break
                else:
                    file1.write('['+'\n')
                while read_skip_record(file, file1):
                    i += 1
                    if not i%print_progress:
                        print("*", end="")
                # ']' on the last line
                file1.write(']'+'\n')
                break

    # if there is a comma at the end, erase it
    if comma: # delete comma after the last observation
        file1.close()
        with open(fire_file_out, 'rb+') as file1:
            file1.seek(-4, os.SEEK_END)
            file1.truncate()
            file1.close()
        with open(fire_file_out, 'a') as file1:
            file1.write('\n]'+'\n')


 Data/fire/fire_nrt_V1_82357.json  ->  Data/fire/fire_nrt_V1_82357_out.json
****************************************************************************************

In [11]:
ls -ltrh ./Data/fire/fire_nrt_M6_82356*

-rw-r--r--  1 bogdan  staff    71M Oct 24 14:56 ./Data/fire/fire_nrt_M6_82356.json
-rw-r--r--  1 bogdan  staff   522K Nov  5 22:02 ./Data/fire/fire_nrt_M6_82356_out.json


In [53]:
json_df = pd.read_json('Data/fire/fire_nrt_V1_82357_out.json')

In [13]:
json_df = pd.read_json('Data/fire/fire_nrt_M6_82356_out.json')

In [17]:
type(json_df)

pandas.core.frame.DataFrame

In [18]:
json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88219 entries, 0 to 88218
Data columns (total 14 columns):
acq_date      88219 non-null object
acq_time      88219 non-null int64
bright_ti4    88219 non-null float64
bright_ti5    88219 non-null float64
confidence    88219 non-null object
daynight      88219 non-null object
frp           88219 non-null float64
instrument    88219 non-null object
latitude      88219 non-null float64
longitude     88219 non-null float64
satellite     88219 non-null object
scan          88219 non-null float64
track         88219 non-null float64
version       88219 non-null object
dtypes: float64(7), int64(1), object(6)
memory usage: 9.4+ MB


In [19]:
json_df.head(15)

,acq_date,acq_time,bright_ti4,bright_ti5,confidence,daynight,frp,instrument,latitude,longitude,satellite,scan,track,version
0,2019-07-01,0,305.5,287.0,n,N,1.9,VIIRS,-7.04908,29.52174,N,0.49,0.65,1.0NRT
1,2019-07-01,0,314.7,268.4,n,N,1.6,VIIRS,13.01929,14.22009,N,0.35,0.56,1.0NRT
2,2019-07-01,0,316.2,290.3,n,N,1.9,VIIRS,-5.57967,27.13916,N,0.32,0.54,1.0NRT
3,2019-07-01,0,316.4,292.1,n,N,2.6,VIIRS,-5.59052,27.07100,N,0.63,0.54,1.0NRT
4,2019-07-01,0,300.3,287.9,n,N,2.4,VIIRS,-2.47963,31.00544,N,0.53,0.67,1.0NRT
5,2019-07-01,6,308.8,287.4,n,N,1.4,VIIRS,-6.78178,22.80324,N,0.52,0.42,1.0NRT
6,2019-07-01,6,306.8,286.9,n,N,1.1,VIIRS,-7.90811,24.53073,N,0.46,0.47,1.0NRT
7,2019-07-01,6,315.4,280.8,n,N,5.2,VIIRS,-25.50902,29.12469,N,0.73,0.76,1.0NRT
8,2019-07-01,6,299.2,286.9,n,N,1.3,VIIRS,-15.36320,29.80463,N,0.65,0.73,1.0NRT
9,2019-07-01,6,313.6,284.8,n,N,1.6,VIIRS,-10.01756,15.63971,N,0.42,0.38,1.0NRT


In [22]:
track=json_df['track'].unique()

In [23]:
track.sort()

In [24]:
track

array([0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44, 0.45, 0.46,
       0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57,
       0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68,
       0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78])

In [25]:
frp=json_df['frp'].unique()

In [26]:
frp[:5]

array([1.9, 1.6, 2.6, 2.4, 1.4])

In [27]:
len(frp)

1496

In [28]:
instrument=json_df['instrument'].unique()

In [29]:
instrument

array(['VIIRS'], dtype=object)

In [30]:
scan=json_df['scan'].unique()

In [31]:
scan

array([0.49, 0.35, 0.32, 0.63, 0.53, 0.52, 0.46, 0.73, 0.65, 0.42, 0.4 ,
       0.54, 0.51, 0.5 , 0.43, 0.48, 0.57, 0.41, 0.62, 0.39, 0.44, 0.47,
       0.45, 0.37, 0.38, 0.67, 0.55, 0.72, 0.58, 0.64, 0.6 , 0.34, 0.33,
       0.61, 0.77, 0.75, 0.79, 0.56, 0.7 , 0.68, 0.36, 0.59, 0.74, 0.76,
       0.71, 0.69, 0.66, 0.78, 0.8 ])

In [32]:
scan.sort()

In [33]:
scan

array([0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42,
       0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53,
       0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64,
       0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75,
       0.76, 0.77, 0.78, 0.79, 0.8 ])

In [34]:
satellite=json_df['satellite'].unique()

In [35]:
satellite

array(['N'], dtype=object)

In [36]:
version=json_df['version'].unique()

In [37]:
version

array(['1.0NRT'], dtype=object)

In [38]:
daynight=json_df['daynight'].unique()

In [39]:
daynight

array(['N', 'D'], dtype=object)

In [40]:
min(json_df['acq_date'])

'2019-07-01'

In [41]:
max(json_df['acq_date'])

'2019-10-24'

In [42]:
len(json_df['acq_date'].unique())

116

In [43]:
confidence=json_df['confidence'].unique()

In [44]:
confidence

array(['n', 'l', 'h'], dtype=object)

In [55]:
fire_files_out = ['fire_archive_M6_82356_out.json', 'fire_archive_M6_82363_out.json',
                  'fire_archive_M6_82479_out.json', 'fire_archive_M6_82481_out.json',
                  'fire_archive_M6_82483_out.json', 'fire_archive_M6_82487_out.json',
                  'fire_archive_M6_82489_out.json', 'fire_archive_M6_82491_out.json',
                  'fire_archive_M6_82493_out.json', 'fire_archive_M6_82495_out.json',
                  'fire_archive_M6_82497_out.json', 'fire_archive_M6_82499_out.json',
                  'fire_archive_M6_82501_out.json', 'fire_archive_M6_82503_out.json',
                  'fire_archive_M6_82505_out.json', 'fire_archive_M6_82508_out.json',
                  'fire_archive_M6_82511_out.json', 'fire_archive_M6_82513_out.json',
                  'fire_archive_M6_82515_out.json', 'fire_archive_M6_82517_out.json',
                  'fire_archive_V1_82357_out.json', 'fire_archive_V1_82364_out.json',
                  'fire_archive_V1_82480_out.json', 'fire_archive_V1_82482_out.json',
                  'fire_archive_V1_82484_out.json', 'fire_archive_V1_82488_out.json',
                  'fire_archive_V1_82490_out.json', 'fire_nrt_M6_82356_out.json',
                  'fire_nrt_V1_82357_out.json']

In [56]:
df_list=[]
for item in fire_files_out:
    print('Loading: ', item, '...')
    df=pd.read_json('Data/fire/' + item)
    df_list.append(df)

Loading:  fire_archive_M6_82356_out.json ...
Loading:  fire_archive_M6_82363_out.json ...
Loading:  fire_archive_M6_82479_out.json ...
Loading:  fire_archive_M6_82481_out.json ...
Loading:  fire_archive_M6_82483_out.json ...
Loading:  fire_archive_M6_82487_out.json ...
Loading:  fire_archive_M6_82489_out.json ...
Loading:  fire_archive_M6_82491_out.json ...
Loading:  fire_archive_M6_82493_out.json ...
Loading:  fire_archive_M6_82495_out.json ...
Loading:  fire_archive_M6_82497_out.json ...
Loading:  fire_archive_M6_82499_out.json ...
Loading:  fire_archive_M6_82501_out.json ...
Loading:  fire_archive_M6_82503_out.json ...
Loading:  fire_archive_M6_82505_out.json ...
Loading:  fire_archive_M6_82508_out.json ...
Loading:  fire_archive_M6_82511_out.json ...
Loading:  fire_archive_M6_82513_out.json ...
Loading:  fire_archive_M6_82515_out.json ...
Loading:  fire_archive_M6_82517_out.json ...
Loading:  fire_archive_V1_82357_out.json ...
Loading:  fire_archive_V1_82364_out.json ...
Loading:  

In [57]:
df_list[0].memory_usage(index = True)

Index             80
acq_date      293896
acq_time      293896
bright_t31    293896
brightness    293896
confidence    293896
daynight      293896
frp           293896
instrument    293896
latitude      293896
longitude     293896
satellite     293896
scan          293896
track         293896
type          293896
version       293896
dtype: int64

In [58]:
df1=df_list[0].memory_usage(index = True)

In [59]:
type(df1)

pandas.core.series.Series

In [60]:
sum(df1)

4408520

In [61]:
sum(df_list[0].memory_usage(index = True))

4408520

In [62]:
len(df_list)

29

In [63]:
df_list_size=[]
for item in df_list:
    df_list_size.append(sum(item.memory_usage(index = True)))

In [64]:
df_list_size

[4408520,
 5074880,
 5342120,
 5358440,
 5665040,
 5117480,
 5539280,
 5678000,
 5702960,
 5363120,
 5806760,
 5868440,
 6140960,
 5674640,
 6164120,
 6108680,
 6206240,
 4490240,
 2156600,
 319040,
 9779808,
 21456368,
 22397392,
 22745488,
 23888784,
 21586064,
 22734288,
 218256,
 9864256]

In [65]:
sum(df_list_size)

256856264

In [66]:
df_list[22].head()

,acq_date,acq_time,bright_ti4,bright_ti5,confidence,frp,instrument,latitude,longitude,satellite,scan,track,type,version
0,2017-01-01,112,328.9,275.7,n,3.8,VIIRS,51.51851,6.97199,N,0.55,0.42,2,1
1,2017-01-01,115,296.9,276.7,n,0.4,VIIRS,38.20984,15.54110,N,0.39,0.44,0,1
2,2017-01-01,116,314.7,280.0,n,1.4,VIIRS,36.86977,6.97846,N,0.40,0.37,2,1
3,2017-01-01,116,296.1,270.6,n,1.1,VIIRS,40.22898,-3.99571,N,0.46,0.64,0,1
4,2017-01-01,117,328.1,289.2,n,2.1,VIIRS,33.89265,12.60565,N,0.51,0.41,3,1


In [ ]:
TODO:
    - decide if acq_time is important or not
    - combine bright_ti4 and bright_ti5 into one feature (brithness temperature from different channels)
    - 

In [105]:
ls -ltrh ./Data/co2

total 152
-rwxrwxrwx  1 bogdan  staff    73K Oct 24 12:16 monthly_in_situ_co2_mlo.csv*


In [121]:
co2_df = pd.read_csv('Data/co2/monthly_in_situ_co2.csv')
#test_df=pd.read_csv('Data/ice/N_seaice_extent_daily_v3.0.csv')

In [117]:
less ./Data/ice/N_seaice_extent_daily_v3.0.csv

In [122]:
co2_df.head(10)

,Yr,mn,Date(Excel),Date,CO2,seasonally_adjusted,fit,seasonally_adjusted_fit,CO2_filled,sesonally_adjusted_filled
0,1958,1,21200,1958.0411,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99
1,1958,2,21231,1958.1260,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99
2,1958,3,21259,1958.2027,315.70,314.43,316.19,314.90,315.70,314.43
3,1958,4,21290,1958.2877,317.45,315.16,317.30,314.98,317.45,315.16
4,1958,5,21320,1958.3699,317.51,314.71,317.86,315.06,317.51,314.71
5,1958,6,21351,1958.4548,-99.99,-99.99,317.24,315.14,317.24,315.14
6,1958,7,21381,1958.5370,315.86,315.19,315.86,315.21,315.86,315.19
7,1958,8,21412,1958.6219,314.93,316.19,313.99,315.29,314.93,316.19
8,1958,9,21443,1958.7068,313.21,316.08,312.46,315.35,313.21,316.08
9,1958,10,21473,1958.7890,-99.99,-99.99,312.44,315.40,312.44,315.40


TODO: 
- deal with -99.99 values
- decide if seasonally adjusted values are ok as features or as outputs
- calculate CO2 for each day for ice_df

In [5]:
ice_north_df = pd.read_csv('Data/ice/N_seaice_extent_daily_v3.0.csv')

In [6]:
ice_north_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13322 entries, 0 to 13321
Data columns (total 6 columns):
Year            13322 non-null object
 Month          13322 non-null object
 Day            13322 non-null object
     Extent     13322 non-null object
    Missing     13322 non-null object
 Source Data    13322 non-null object
dtypes: object(6)
memory usage: 624.6+ KB


In [7]:
ice_north_df.head()

,Year,Month,Day,Extent,Missing,Source Data
0,YYYY,MM,DD,10^6 sq km,10^6 sq km,Source data product web sites: http://nsidc.o...
1,1978,10,26,10.231,0.000,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...
2,1978,10,28,10.420,0.000,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...
3,1978,10,30,10.557,0.000,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...
4,1978,11,01,10.670,0.000,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...


In [10]:
ice_north_df.columns

Index(['Year', ' Month', ' Day', '     Extent', '    Missing', ' Source Data'], dtype='object')

In [11]:
# drop useless columns
ice_north_df.drop(['    Missing', ' Source Data'], axis=1, inplace=True)

In [14]:
ice_north_df.columns

Index(['Year', ' Month', ' Day', '     Extent'], dtype='object')

In [12]:
ice_south_df = pd.read_csv('Data/ice/S_seaice_extent_daily_v3.0.csv')

In [13]:
ice_south_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13322 entries, 0 to 13321
Data columns (total 6 columns):
Year            13322 non-null object
 Month          13322 non-null object
 Day            13322 non-null object
     Extent     13322 non-null object
    Missing     13322 non-null object
 Source Data    13322 non-null object
dtypes: object(6)
memory usage: 624.6+ KB


In [15]:
ice_south_df.columns

Index(['Year', ' Month', ' Day', '     Extent', '    Missing', ' Source Data'], dtype='object')

In [26]:
# There are a few records for 'S_seaice_extent_daily_v3.0.csv' but the missing values are less than 0.001
# These missing values will be ignored
ice_south_df[1::]['    Missing'].astype(float)[ice_south_df[1::]['    Missing'].astype(float)!=0.]

13040    0.001
13188    0.001
13214    0.001
13302    0.002
13310    0.001
Name:     Missing, dtype: float64

In [27]:
# drop useless columns
ice_south_df.drop(['    Missing', ' Source Data'], axis=1, inplace=True)

In [28]:
ice_south_df.columns

Index(['Year', ' Month', ' Day', '     Extent'], dtype='object')

In [30]:
'1978-10-30' < '1978-11-01'

True

In [43]:
ice_south_df.iloc[1][0].strip()+'-'+ice_south_df.iloc[1][1].strip()+'-'+ice_south_df.iloc[1][2].strip()

'1978-10-26'

In [56]:
#ice_south_df.iloc[:,0].str.strip()+'-'+ice_south_df.iloc[:,1].str.strip()+'-'+ice_south_df.iloc[:,2].str.strip()
ice_south_df['acq_date'] = ice_south_df.iloc[:,0].str.strip() + '-' + ice_south_df.iloc[:,1].str.strip() + '-' + ice_south_df.iloc[:,2].str.strip()

In [80]:
ice_south_df.head()

,Year,Month,Day,Extent,acq_date
1,1978,10,26,17.624,1978-10-26
2,1978,10,28,17.803,1978-10-28
3,1978,10,30,17.670,1978-10-30
4,1978,11,01,17.527,1978-11-01
5,1978,11,03,17.486,1978-11-03


In [79]:
ice_south_df.drop(0, inplace=True)

In [81]:
ice_south_df['extent_south'] = ice_south_df.iloc[:,3].str.strip().astype(float)

In [84]:
ice_south_df.head()

,acq_date,extent_south
1,1978-10-26,17.624
2,1978-10-28,17.803
3,1978-10-30,17.670
4,1978-11-01,17.527
5,1978-11-03,17.486


In [83]:
ice_south_df = ice_south_df.drop(ice_south_df.columns[[0, 1, 2, 3]], axis=1)

In [88]:
ice_south_df.set_index('acq_date', inplace=True)

In [89]:
ice_south_df.head()

,extent_south
acq_date,
1978-10-26,17.624
1978-10-28,17.803
1978-10-30,17.670
1978-11-01,17.527
1978-11-03,17.486


In [90]:
ice_south_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13321 entries, 1978-10-26 to 2019-10-22
Data columns (total 1 columns):
extent_south    13321 non-null float64
dtypes: float64(1)
memory usage: 208.1+ KB


In [94]:
type(ice_south_df['extent_south'][4])

numpy.float64

In [95]:
ice_north_df.columns

Index(['Year', ' Month', ' Day', '     Extent'], dtype='object')

In [96]:
ice_north_df['acq_date'] = ice_north_df.iloc[:,0].str.strip() + '-' + ice_north_df.iloc[:,1].str.strip() + '-' + ice_north_df.iloc[:,2].str.strip()

In [97]:
ice_north_df.head()

,Year,Month,Day,Extent,acq_date
0,YYYY,MM,DD,10^6 sq km,YYYY-MM-DD
1,1978,10,26,10.231,1978-10-26
2,1978,10,28,10.420,1978-10-28
3,1978,10,30,10.557,1978-10-30
4,1978,11,01,10.670,1978-11-01


In [98]:
ice_north_df.drop(0, inplace=True)

In [101]:
ice_north_df['extent_north'] = ice_north_df.iloc[:,3].str.strip().astype(float)

In [102]:
ice_north_df.head()

,Year,Month,Day,Extent,acq_date,extent_south,extent_north
1,1978,10,26,10.231,1978-10-26,10.231,10.231
2,1978,10,28,10.420,1978-10-28,10.420,10.420
3,1978,10,30,10.557,1978-10-30,10.557,10.557
4,1978,11,01,10.670,1978-11-01,10.670,10.670
5,1978,11,03,10.777,1978-11-03,10.777,10.777


In [103]:
ice_north_df = ice_north_df.drop(ice_north_df.columns[[0, 1, 2, 3]], axis=1)

In [104]:
ice_north_df.head()

,acq_date,extent_north
1,1978-10-26,10.231
2,1978-10-28,10.420
3,1978-10-30,10.557
4,1978-11-01,10.670
5,1978-11-03,10.777


In [105]:
ice_north_df.set_index('acq_date', inplace=True)

In [107]:
ice_north_df.head()

,extent_north
acq_date,
1978-10-26,10.231
1978-10-28,10.420
1978-10-30,10.557
1978-11-01,10.670
1978-11-03,10.777


In [108]:
ice_north_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13321 entries, 1978-10-26 to 2019-10-22
Data columns (total 1 columns):
extent_north    13321 non-null float64
dtypes: float64(1)
memory usage: 208.1+ KB


In [109]:
ice_south_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13321 entries, 1978-10-26 to 2019-10-22
Data columns (total 1 columns):
extent_south    13321 non-null float64
dtypes: float64(1)
memory usage: 848.1+ KB


In [110]:
ice_df = pd.concat([ice_north_df, ice_south_df], axis=1)

In [111]:
ice_df

,extent_north,extent_south
acq_date,,
1978-10-26,10.231,17.624
1978-10-28,10.420,17.803
1978-10-30,10.557,17.670
1978-11-01,10.670,17.527
1978-11-03,10.777,17.486
...,...,...
2019-10-18,5.539,17.917
2019-10-19,5.651,17.840
2019-10-20,5.687,17.737


In [112]:
ice_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13321 entries, 1978-10-26 to 2019-10-22
Data columns (total 2 columns):
extent_north    13321 non-null float64
extent_south    13321 non-null float64
dtypes: float64(2)
memory usage: 312.2+ KB


In [113]:
ice_df.describe()

,extent_north,extent_south
count,13321.000000,13321.000000
mean,11.312060,11.677718
std,3.287966,5.640277
min,3.340000,2.080000
25%,8.573000,6.072000
50%,12.010000,12.634000
75%,14.220000,17.241000
max,16.635000,20.201000


In [11]:
ice_regional_df = pd.ExcelFile('Data/ice/Sea_Ice_Index_Regional_Daily_Data_G02135_v3.0.xlsx')

In [12]:
ice_regional_df.sheet_names

['Baffin-Area-km^2',
 'Baffin-Extent-km^2',
 'Barents-Area-km^2',
 'Barents-Extent-km^2',
 'Beaufort-Area-km^2',
 'Beaufort-Extent-km^2',
 'Bering-Area-km^2',
 'Bering-Extent-km^2',
 'CanadianArchipelago-Area-km^2',
 'CanadianArchipelago-Extent-km^2',
 'Central-Arctic-Area-km^2',
 'Central-Arctic-Extent-km^2',
 'Chukchi-Area-km^2',
 'Chukchi-Extent-km^2',
 'East-Siberian-Area-km^2',
 'East-Siberian-Extent-km^2',
 'Greenland-Area-km^2',
 'Greenland-Extent-km^2',
 'Hudson-Area-km^2',
 'Hudson-Extent-km^2',
 'Kara-Area-km^2',
 'Kara-Extent-km^2',
 'Laptev-Area-km^2',
 'Laptev-Extent-km^2',
 'Okhotsk-Area-km^2',
 'Okhotsk-Extent-km^2',
 'St-Lawrence-Area-km^2',
 'St-Lawrence-Extent-km^2',
 'Documentation']

In [13]:
ice_regional_df.parse(1)

,month,day,1978,1979,1980,1981,1982,1983,1984,1985,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,January,1,NaN,903186.035,1013769.883,1033270.716,1023094.590,1347723.970,1309901.559,1341199.586,...,795608.107,657280.210,970785.241,836904.594,1042165.178,919851.203,1107737.558,1043957.619,852411.168,917527.054
1,NaN,2,NaN,928997.908,999873.492,1050801.408,1021066.245,1372848.377,1350548.340,1360997.397,...,790348.007,665528.563,988757.101,846580.138,1045929.234,952044.039,1118000.089,1041095.765,872204.561,942891.608
2,NaN,3,NaN,947432.787,1000897.098,1041880.956,1013746.335,1417371.453,1372754.604,1376789.972,...,786922.827,670551.044,999673.091,853340.629,1044428.577,984973.384,1126274.372,1039445.720,891754.362,961108.705
3,NaN,4,NaN,974928.335,997687.620,1037514.096,1030366.773,1460409.917,1392297.001,1398633.692,...,784717.294,670551.024,1006333.892,863309.928,1047832.206,1008812.804,1131821.502,1040792.446,912064.230,985959.419
4,NaN,5,NaN,1005270.542,1015031.822,1018820.331,1040308.602,1491475.904,1431611.850,1421457.076,...,787263.871,668051.543,1010444.124,877502.215,1047953.087,1031026.911,1137901.759,1055976.206,926768.633,1002583.776
5,NaN,6,NaN,1020866.519,1026332.269,1002736.428,1039142.313,1530012.742,1441697.272,1425325.650,...,783989.956,666964.802,1012726.638,896387.766,1045552.610,1041425.668,1143122.799,1073472.421,928374.126,1019351.792
6,NaN,7,NaN,1040988.952,1069400.637,1010485.255,1050208.692,1531359.065,1446624.956,1437691.966,...,784935.765,659703.793,1012993.471,918865.668,1051147.246,1051546.883,1146589.922,1077058.157,921234.302,1030276.454
7,NaN,8,NaN,1064087.127,1102628.800,1003707.856,1033289.669,1543829.795,1439200.445,1416160.241,...,792010.474,652593.179,1020406.891,937745.734,1054370.214,1067754.241,1147676.338,1081727.365,913151.686,1041308.047
8,NaN,9,NaN,1081170.976,1108456.284,1032386.264,1018130.679,1559868.703,1443109.770,1403079.796,...,800453.302,651912.342,1037356.960,961546.402,1054033.087,1090845.365,1147695.498,1082576.027,906364.321,1045206.417
9,NaN,10,NaN,1094338.710,1137824.314,1057862.996,1019872.450,1562999.115,1442072.753,1393712.912,...,811319.869,655219.873,1055413.543,979023.324,1063678.220,1107823.450,1154062.400,1087178.770,908123.416,1053270.988


In [14]:
baffin_extent_df=ice_regional_df.parse('Baffin-Extent-km^2')

In [15]:
baffin_extent_df.columns

Index(['month',   'day',    1978,    1979,    1980,    1981,    1982,    1983,
          1984,    1985,    1986,    1987,    1988,    1989,    1990,    1991,
          1992,    1993,    1994,    1995,    1996,    1997,    1998,    1999,
          2000,    2001,    2002,    2003,    2004,    2005,    2006,    2007,
          2008,    2009,    2010,    2011,    2012,    2013,    2014,    2015,
          2016,    2017,    2018,    2019],
      dtype='object')

In [16]:
len(baffin_extent_df.columns)

44

In [17]:
df_header=baffin_extent_df.columns

In [18]:
type(df_header[2])

int

In [19]:
for i in range(2, len(df_header)):
    print(str(df_header[i]))

1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [20]:
baffin_extent_dates_df=baffin_extent_df[['month', 'day']]

In [35]:
baffin_extent_dates_df.head(35)

,month,day
0,January,1
1,NaN,2
2,NaN,3
3,NaN,4
4,NaN,5
5,NaN,6
6,NaN,7
7,NaN,8
8,NaN,9
9,NaN,10


In [21]:
len(baffin_extent_dates_df['month'])

366

In [22]:
test_df=baffin_extent_df.iloc[:,0:4]

In [23]:
test_df

,month,day,1978,1979
0,January,1,NaN,903186.035
1,NaN,2,NaN,928997.908
2,NaN,3,NaN,947432.787
3,NaN,4,NaN,974928.335
4,NaN,5,NaN,1005270.542
5,NaN,6,NaN,1020866.519
6,NaN,7,NaN,1040988.952
7,NaN,8,NaN,1064087.127
8,NaN,9,NaN,1081170.976
9,NaN,10,NaN,1094338.710


In [24]:
test_df.iloc[:,3]

0       903186.035
1       928997.908
2       947432.787
3       974928.335
4      1005270.542
5      1020866.519
6      1040988.952
7      1064087.127
8      1081170.976
9      1094338.710
10     1115478.828
11     1111839.693
12     1112617.801
13     1124565.065
14     1126510.508
15     1152426.282
16     1176358.712
17     1169864.835
18     1180567.455
19     1166321.412
20     1147353.203
21     1139970.065
22     1131516.557
23     1098805.043
24     1079292.901
25     1062813.539
26     1031618.415
27     1042240.105
28     1046669.150
29     1051752.985
          ...     
336     767211.836
337     781456.741
338     791850.710
339     801800.774
340     805156.889
341     812253.885
342     806301.176
343     803132.440
344     803631.364
345     799512.485
346     820981.898
347     834274.968
348     861868.970
349     890488.850
350     922441.723
351     951702.223
352     948435.230
353     964124.358
354     972623.711
355     965761.831
356     996482.484
357    10237

In [25]:
months=['01','02','03','04','05','06','07','08','09','10','11','12']
i=-1
month_str=[]
for item in test_df['month']:
    if type(item)==type(""): # start of new month, the series starts with "January"
        i += 1
    month_str.append(months[i])

In [26]:
day_str=[]
for item in test_df['day']:
    day_str.append(str(item).zfill(2))

In [27]:
month_day_str=[]
for i in range(len(month_str)):
    month_day_str.append(month_str[i]+'-'+day_str[i])

In [28]:
test_df['month_day_str']=month_day_str

In [29]:
test_df

,month,day,1978,1979,month_day_str
0,January,1,NaN,903186.035,01-01
1,NaN,2,NaN,928997.908,01-02
2,NaN,3,NaN,947432.787,01-03
3,NaN,4,NaN,974928.335,01-04
4,NaN,5,NaN,1005270.542,01-05
5,NaN,6,NaN,1020866.519,01-06
6,NaN,7,NaN,1040988.952,01-07
7,NaN,8,NaN,1064087.127,01-08
8,NaN,9,NaN,1081170.976,01-09
9,NaN,10,NaN,1094338.710,01-10


In [31]:
test_df.columns

Index(['month', 'day', 1978, 1979, 'month_day_str'], dtype='object')

In [34]:
test_df.drop(test_df.columns[[0, 1]], axis=1, inplace=True)
test_df.columns
#df = df.drop(df.columns[[0, 1, 3]], axis=1, inplace=True)

Index([1978, 1979, 'month_day_str'], dtype='object')

In [35]:
test_df

,1978,1979,month_day_str
0,NaN,903186.035,01-01
1,NaN,928997.908,01-02
2,NaN,947432.787,01-03
3,NaN,974928.335,01-04
4,NaN,1005270.542,01-05
5,NaN,1020866.519,01-06
6,NaN,1040988.952,01-07
7,NaN,1064087.127,01-08
8,NaN,1081170.976,01-09
9,NaN,1094338.710,01-10


In [38]:
test_df.set_index('month_day_str', inplace=True)

In [39]:
type(test_df.index)

pandas.core.indexes.base.Index

In [46]:
test_df

,1978,1979
month_day_str,,
01-01,NaN,903186.035
01-02,NaN,928997.908
01-03,NaN,947432.787
01-04,NaN,974928.335
01-05,NaN,1005270.542
01-06,NaN,1020866.519
01-07,NaN,1040988.952
01-08,NaN,1064087.127
01-09,NaN,1081170.976


In [49]:
for item in test_df.index:
    print(item)

01-01
01-02
01-03
01-04
01-05
01-06
01-07
01-08
01-09
01-10
01-11
01-12
01-13
01-14
01-15
01-16
01-17
01-18
01-19
01-20
01-21
01-22
01-23
01-24
01-25
01-26
01-27
01-28
01-29
01-30
01-31
02-01
02-02
02-03
02-04
02-05
02-06
02-07
02-08
02-09
02-10
02-11
02-12
02-13
02-14
02-15
02-16
02-17
02-18
02-19
02-20
02-21
02-22
02-23
02-24
02-25
02-26
02-27
02-28
02-29
03-01
03-02
03-03
03-04
03-05
03-06
03-07
03-08
03-09
03-10
03-11
03-12
03-13
03-14
03-15
03-16
03-17
03-18
03-19
03-20
03-21
03-22
03-23
03-24
03-25
03-26
03-27
03-28
03-29
03-30
03-31
04-01
04-02
04-03
04-04
04-05
04-06
04-07
04-08
04-09
04-10
04-11
04-12
04-13
04-14
04-15
04-16
04-17
04-18
04-19
04-20
04-21
04-22
04-23
04-24
04-25
04-26
04-27
04-28
04-29
04-30
05-01
05-02
05-03
05-04
05-05
05-06
05-07
05-08
05-09
05-10
05-11
05-12
05-13
05-14
05-15
05-16
05-17
05-18
05-19
05-20
05-21
05-22
05-23
05-24
05-25
05-26
05-27
05-28
05-29
05-30
05-31
06-01
06-02
06-03
06-04
06-05
06-06
06-07
06-08
06-09
06-10
06-11
06-12
06-13
06-14
06-1

In [50]:
test_df.columns

Index([1978, 1979], dtype='object')

In [58]:
for year in test_df.columns:
    print(str(year))

1978
1979


In [59]:
test1_df=pd.DataFrame()

In [62]:
test1_df

""


In [64]:
ice_date=[]

In [65]:
for year in test_df.columns:
    for month_day in test_df.index:
        ice_date.append(str(year)+'-'+month_day)

In [67]:
test1_df['date']=ice_date

In [85]:
test1_df.drop(test1_df.columns[1], axis=1, inplace=True)

In [86]:
test1_df.columns

Index(['date'], dtype='object')

In [87]:
test1_df

,date
0,1978-01-01
1,1978-01-02
2,1978-01-03
3,1978-01-04
4,1978-01-05
5,1978-01-06
6,1978-01-07
7,1978-01-08
8,1978-01-09
9,1978-01-10
